This notebook is a tutorial for ATAC analysis using SEACells and includes computation of gene-peak associations, ATAC gene scores, gene accessibility scores and identification of highly regulated genes

# Imports

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
import SEACells

findfont: Font family ['Raleway'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Lato'] not found. Falling back to DejaVu Sans.


In [3]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Some plotting aesthetics
%matplotlib inline

sns.set_style('ticks')
matplotlib.rcParams['figure.figsize'] = [4, 4]
matplotlib.rcParams['figure.dpi'] = 100


# Load Data

We recommend the use of scanpy Anndata objects as the preferred mode of loading and filtering data.

A sample datset is available for download with the instructions listed below. This is a filtered, unnormalized counts of multiome dataset of CD34+ sorted bone marrow cells to profile human hematopoiesis [Dataset ref TBD]. 

Uncomment the following lines to download the sample dataset in a Unix-based system. For non-UNIX systems, download the files using the URL

In [5]:
# !mkdir data/
# !wget https://dp-lab-data-public.s3.amazonaws.com/SEACells-multiome/cd34_multiome_rna.h5ad -O data/cd34_multiome_rna.h5ad # RNA data
# !wget https://dp-lab-data-public.s3.amazonaws.com/SEACells-multiome/cd34_multiome_atac.h5ad -O data/cd34_multiome_atac.h5ad # ATAC data

The dataset contains RNA and ATAC modalities as two different Anndata objects. The ATAC dataset contains precomputed SEACell metacells 

In [6]:
#import os
#os.listdir('/omics/groups/OE0533/internal/katharina/scDoRI/gastrulation_data/jupyter_notebooks/data/')

FileNotFoundError: [Errno 2] No such file or directory: 'data/'

In [9]:
# Load the data using scanpy
rna_ad = sc.read('/omics/groups/OE0533/internal/katharina/scDoRI/gastrulation_data/data/cd34_multiome_rna.h5ad')
atac_ad = sc.read('/omics/groups/OE0533/internal/katharina/scDoRI/gastrulation_data/jupyter_notebooks/archr_.h5ad')

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/omics/groups/OE0533/internal/katharina/scDoRI/gastrulation_data/data/cd34_multiome_atac.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
rna_ad

In [ ]:
atac_ad

In [ ]:
# Plot cell-types for reference (RNA)
sc.pl.scatter(rna_ad, basis='umap', color='celltype', frameon=False)

In [ ]:
# Plot cell-types for reference (ATAC)
sc.pl.scatter(atac_ad, basis='umap', color='celltype', frameon=False)

In [ ]:
SEACells.plot.plot_2D(atac_ad, key='X_umap', colour_metacells=True)

# Preparation step


In the first step, we derive summarized ATAC and RNA SEACell metacells Anndata objects. Both the input single-cell RNA and ATAC anndata objects should contain raw, unnormalized data. SEACell results on ATAC data will be used for the summarization

<b>Warning: </b> The ATAC and RNA single-cell Anndata objects should contain the same set of cells. Only the common cells will be used for downstream analyses.

In [ ]:
atac_meta_ad, rna_meta_ad = SEACells.genescores.prepare_multiome_anndata(atac_ad, rna_ad, SEACell_label='SEACell')

The preparation step will generate summarized anndata objects for RNA and ATAC

In [ ]:
atac_meta_ad

In [ ]:
rna_meta_ad

# Gene-peak correlations

Using the paired multiome metacell data, the next step is to compute the correlation of gene expression and accessbility of peaks within the vicinity of the gene. 

Computation of gene peak correlations requires the following parameters :
1. GTF file with gene annotations. <b> Note: </b> Chromosome names should be numbered 1, 2 and the "chr" prefix will be added by SEACells 
2. Genomic span around genes to test the correlations 

Human GTF file is available at [https://dp-lab-data-public.s3.amazonaws.com/SEACells-multiome/hg38.gtf](https://dp-lab-data-public.s3.amazonaws.com/SEACells-multiome/hg38.gtf) and can be downloaded using:

In [ ]:
#! wget https://dp-lab-data-public.s3.amazonaws.com/SEACells-multiome/hg38.gtf -O data/hg38.gtf

In [ ]:
# In this example, we compute gene peak correlations for the first ten genes since 
# this process is computationally intensive
gene_set = rna_meta_ad.var_names[:10]
gene_peak_cors = SEACells.genescores.get_gene_peak_correlations(atac_meta_ad, rna_meta_ad, 
                                           path_to_gtf='data/hg38.gtf', 
                                           span=100000, 
                                           n_jobs=1,
                                           gene_set=gene_set)

The result of this function is a `pandas.Series` object with one entry for each gene. Each entry is a `pandas.DataFrame` with the correlation of peak accessibility and gene expression and the p-value for significance of correlation using GC and accessiblity matched background sets

In [ ]:
gene_peak_cors['FAM41C'].head()

If no peaks are present in the specified span of the gene, the entry in the dictionary contains a zero

In [ ]:
gene_peak_cors['LINC01128']

## Highly regulated genes

For downstream analyses, the full gene peak correlations results are available as a pickle file at [https://dp-lab-data-public.s3.amazonaws.com/SEACells-multiome/cd34_multiome_gene_peak_cors.p](https://dp-lab-data-public.s3.amazonaws.com/SEACells-multiome/cd34_multiome_gene_peak_cors.p) and can be downloaded using


In [ ]:
#! wget https://dp-lab-data-public.s3.amazonaws.com/SEACells-multiome/cd34_multiome_gene_peak_cors.p -O data/cd34_multiome_gene_peak_cors.p

In [ ]:
gene_peak_cors = pd.read_pickle('data/cd34_multiome_gene_peak_cors.p') 

In [ ]:
len(gene_peak_cors)

Highly regulated genes i.e., genes that are correlated with multiple peaks can be identified using the `get_gene_peak_assocations` function. `get_gene_peak_assocations` returns the number of significantly peaks correlated with each gene

In [ ]:
peak_counts = SEACells.genescores.get_gene_peak_assocations(gene_peak_cors, 
                                                           pval_cutoff=1e-1,
                                                           cor_cutoff=0.1)

In [ ]:
peak_counts

In [ ]:
# Plot the distribution to identify genes with higher degree of regulation
plt.scatter(np.arange(len(peak_counts)), 
           np.sort(peak_counts), s=20)
sns.despine()
plt.xlabel('Gene rank')
plt.ylabel('No. of correlated peaks')

## Gene scores

Gene scores are computed as the weighted sum of the accessiblity of correlated peaks and can be computed using `get_gene_scores`. 

In [ ]:
gene_scores = SEACells.genescores.get_gene_scores(atac_meta_ad, 
                                                  gene_peak_cors,
                                                  pval_cutoff=1e-1,
                                                  cor_cutoff=0.1)

In [ ]:
gene_scores.head()

In [ ]:
gene_scores.shape

`gene_scores` is a `pandas.DataFrame` with metacells as rows and genes as columns. This can be used for any downstream analysis such as clustering, visualization etc.

# Gene-accessibility

This section describes how to compute gene accessiblity metrics using SEACell metacells.

## Open peaks in metacells

The first step is to identify the subset of peaks that are open in each metacell. `determine_metacell_open_peaks` function can be used to determine this. 

By default, all peaks are tested to check if they are open or closed in every metacell. A subset of peaks can be specified using the `peak_set` parameter. 

This function also requires a low-dimensional embedding such as `X_svd`. We can summarize the SVD of single-cell ATAC for this analysis

In [ ]:
# Create a metacell anndata with raw counts
atac_meta_ad = SEACells.core.summarize_by_SEACell(atac_ad, SEACells_label='SEACell')
atac_meta_ad.obs['n_counts'] = np.ravel(atac_meta_ad.X.sum(axis=1))

In [ ]:
# We will reuse the atac_meta_ad computed above

# Add SVD summary to atac meta ad
seacell_label = 'SEACell'
sc_svd = pd.DataFrame(atac_ad.obsm['X_svd'], index=atac_ad.obs_names)
atac_meta_ad.obsm['X_svd'] = sc_svd.groupby(atac_ad.obs[seacell_label]).mean().loc[atac_meta_ad.obs_names, :]

In [ ]:
# Determine open peaks in each metacell
SEACells.accessibility.determine_metacell_open_peaks(atac_meta_ad, peak_set=None, low_dim_embedding='X_svd', pval_cutoff=1e-2,
                                  read_len=147, n_neighbors=3, n_jobs=1)
# This function will add 'OpenPeaks' to the Anndata layers and is a binary matrix 
# indicating whether the peak is open or closed in the metacell

In [ ]:
atac_meta_ad

## Gene accessibility metric

Open peaks are used to compute gene accessiblity metric which represents the fraction of correlated open peaks. 

<b>Warning: </b> This metric is only reliable if there are sufficient number of open peaks associated with each gene. 
It is recommended to be used for only genes with high regulation

In [ ]:
# Use the highly regulated genes as the gene set of interest 
# Plot the distribution to identify genes with higher degree of regulation
plt.scatter(np.arange(len(peak_counts)), 
           np.sort(peak_counts), s=20)
sns.despine()
plt.xlabel('Gene rank')
plt.ylabel('No. of correlated peaks')

In [ ]:
# Select genes based on the elbow point
high_reg_genes = peak_counts.index[peak_counts > 9]

In [ ]:
# Compute gene accessibility
SEACells.accessibility.get_gene_accessibility(atac_meta_ad, gene_peak_cors, 
                                              gene_set=high_reg_genes, pval_cutoff=1e-1, cor_cutoff=0.1)
# p-value and corrrelation cutoffs are used for correlated peaks
# This function will add 'GeneAccessibility' to the Anndata `.obsm` field

In [ ]:
atac_meta_ad.obsm['GeneAccessibility']

Gene accessiblity metrics can be used as inputs for downstream analyses.

### Visualization

In [ ]:
# First generate a summarized umap to visualize gene accessilibility
# We will use the RNA meta data to compare expression and accessibility 
rna_umap = pd.DataFrame(rna_ad.obsm['X_umap'], index=rna_ad.obs_names)
rna_meta_ad.obsm['X_umap'] = rna_umap.groupby(atac_ad.obs[seacell_label]).mean().loc[rna_meta_ad.obs_names, :].values


In [ ]:
genes = ['KLF1', 'GATA1', 'SPI1']
# Copy accessibility to RNA meta anndata
temp = rna_meta_ad[:, genes]
temp.layers['GeneAccessibility'] = atac_meta_ad[rna_meta_ad.obs_names].obsm['GeneAccessibility'][genes].values

# Plot expression
sc.pl.scatter(rna_meta_ad, basis='umap', color=genes)

# Plot accessibility
sc.pl.scatter(temp, basis='umap', color=genes, layers='GeneAccessibility')